# Neural information retrieval

In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import spacy

/home/vojta/anaconda3/envs/kaggle/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/home/vojta/anaconda3/envs/kaggle/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## EDA

In [2]:
chairs_df = pd.read_csv('../data/kancelarska-kresla.heureka.cz.csv')

In [3]:
chairs_df.sample(5)

,name,categories,price,rating,specification
228,Neoseat Kiddy Three,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",3480,96.0,Dětská rostoucí židle pro školáka s odolným te...
345,ADK Trade Basic,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",1490,91.0,Kvalitní kancelářská židle ADK Basic nabízí ne...
386,Sedia 8 CP,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",1960,100.0,Nosnost 120kg. Výběr potahu. Výběr provedení.
380,Casaria 100937,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",1750,97.0,"Sezení, které šetří záda díky ergonomicky tvar..."
701,Rauman Soren,"['Bydlení a doplňky', 'Nábytek', 'Židle a stol...",10889,97.0,Kancelářská židle Soren Židle Soren si zaslouž...


In [4]:
chairs_df.isnull().sum()

name              0
categories        0
price             0
rating           13
specification     6
dtype: int64

In [5]:
# replace missing values of rating with mean
chairs_df['rating'] = chairs_df['rating'].fillna(chairs_df['rating'].mean())

In [6]:
# replace missing specification with empty string
chairs_df['specification'] = chairs_df['specification'].fillna('')

In [7]:
chairs_df['name'] = chairs_df['name'].astype(str)
chairs_df['specification'] = chairs_df['specification'].astype(str)

## Model

In [19]:
def transform_query(query: str) -> str:
    '''
    Transforms query into a form that can be used for searching relevant passages.

    Args:
    query: str: query string

    Returns:
    str: transformed query string
    '''
    return f'Represent this sentence for searching relevant passages: {query}'

backbone = 'mixedbread-ai/mxbai-embed-large-v1'
model = SentenceTransformer(backbone)

In [20]:
query = 'Ergonomické křeslo.'
query = transform_query(query)
docs = [query]
sep_token = '###'

for i in range(10):
    price = chairs_df["price"].iloc[i]
    price_cat = ''
    if price <= 1000:
        price_cat = 'very cheap'
    elif price <= 5000:
        price_cat = 'cheap'
    elif price <= 10000:
        price_cat = 'expensive'
    else:
        price_cat = 'very expensive'
    tmp = f'Jméno: {chairs_df["name"].iloc[i]} {sep_token} Cena: {price_cat} {sep_token} Popis: {chairs_df["specification"].iloc[i]}'
    docs.append(tmp)

print(f'Docs: {docs}')

embeddings = model.encode(docs)

similarities = cos_sim(embeddings[0], embeddings[1:])
print('similarities:', similarities)

Docs: ['Represent this sentence for searching relevant passages: Ergonomické křeslo.', 'Jméno: Superkancl Optima ### Cena: cheap ### Popis: Kancelářská židle OPTIMA PDH s područkami a podhlavníkem má vše pro pohodlné sezení. Její předností je široký sedák, ten je čalouněný kvalitní prodyšnou látkou. Výplň sedáku je z pěny s vysokou hustotou a odolností proti prosezení. Ani při delším sezení se nekroutí a umožňuje tak optimální komfort sezení. Opěradlo zad s prodyšným potahem ze síťoviny podpírá páteř a díky svému ergonomickému tvaru předchází bolestem zad. Je doplněné o bederní opěrku a zakončené pevným podhlavníkem. Ruce si můžete pohodlně položit na čalouněné područky, které se dají odklopit směrem nahoru. Kancelářskou židli tak snadno zasunete pod stůl. Svojí velikostí je vhodná pro osoby s výškou do 185 cm. Židle má kvalitní houpací mechanismus s nastavením síly houpání, ta se reguluje velkým plastovým šroubem umístěným pod sedákem. Výška sezení se upravuje plynule páčkou. Je pou

In [21]:
type(embeddings)

numpy.ndarray

## SpaCy testing

[('Křeslo', 'LOC')]
